#### Here, we are developing our Stock Market Long/Short RSI strategy

Specifications: 
* Trading on 2-minute candles on stocks which have high liquidity and are shortable 
* Going long when the RSI indicator reaches "oversold" conditions (RSI <= $r_s$) 
* Going short when the RSI indciator reaches "overbought" conditions (RSI >= $r_b$)

The exit conditions are: 
* Stop loss -> sized to daily volatility based on EWM model of daily returns scaled down to intraday vol
* Time horizon -> not holding position for longer than 1 hour
* Take profit -> we take profit on the other side of the stop loss level

In [1]:
import pandas as pd 
import numpy as np 
import pandas_ta as ta
import matplotlib.pyplot as plt

from datetime import datetime, timedelta, date
from telegrambot import telegram_bot_keys, TelegramBot
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrameUnit
from alpaca.data.timeframe import TimeFrame
from alpaca.trading.requests import MarketOrderRequest, GetCalendarRequest
from alpaca.trading.enums import OrderSide, TimeInForce
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.trading.client import TradingClient
from alpaca.trading.models import Calendar
from typing import List, Union
from utils import filter_trading_day, check_if_market_open, get_market_time
from load_api_keys import alpaca_paper_trading_keys, load_alpaca_paper_trading_keys


In [3]:
class StocksRSILongShortStrategy: 

    def __init__(self, symbol: str, timeframe: TimeFrame, rsi_overbought_level: int, rsi_oversold_level: int, stop_loss_sigma: float, take_profit_sigma: float, timeout: pd.Timedelta, telegram_keys: telegram_bot_keys, alpaca_paper_trading_keys: alpaca_paper_trading_keys) -> None:
        self.symbol = symbol
        self.timeframe = timeframe
        self.rsi_overbought_level = rsi_overbought_level
        self.rsi_oversold_level = rsi_oversold_level
        self.stop_loss_sigma = stop_loss_sigma
        self.take_profit_sigma = take_profit_sigma
        self.timeout = timeout
        self.telegram_keys = telegram_keys
        self.alpaca_paper_trading_keys = alpaca_paper_trading_keys

        self.telegram_bot = TelegramBot.from_telegram_keys(telegram_keys=self.telegram_keys)
        self.trading_client = TradingClient(
            api_key=self.alpaca_paper_trading_keys.api_key, 
            secret_key=self.alpaca_paper_trading_keys.secret_key
        )

        self.historical_data_client = StockHistoricalDataClient(
            api_key=self.alpaca_paper_trading_keys.api_key, 
            secret_key=self.alpaca_paper_trading_keys.secret_key
        )

        self.current_position = None 
        self.last_trade_time: pd.Timestamp = None 
        self.trades = []
        

    def _get_historical_data(self) -> pd.DataFrame: 
        data_request = StockBarsRequest(
            symbol_or_symbols=self.symbol, 
            start = datetime.now() - timedelta(days = 1), 
            end = datetime.now(), 
            timeframe = self.timeframe 
        )

        data = self.historical_data_client.get_stock_bars(request_params=data_request)

        return data.df.droplevel(level = 0)

    def _get_rsi(self, column: str = 'close'): 

        historical_data = self._get_historical_data()
        rsi_series = ta.rsi(close = historical_data[column]).dropna()

        return rsi_series.iloc[-1]

    

Going through the process of doing calculations and what not

In [4]:
import os

paper_trading_keys = load_alpaca_paper_trading_keys(dotenv_path=r'C:\Users\srerr\Documents\Projects\PersonalProjects\stonks\alpaca\.env')

In [5]:
trading_client = TradingClient(api_key=paper_trading_keys.api_key, secret_key=paper_trading_keys.secret_key, paper=True)
historical_data_client = StockHistoricalDataClient(api_key=paper_trading_keys.api_key, secret_key=paper_trading_keys.secret_key)

In [6]:
timeframe = TimeFrame(
    amount = 2, 
    unit = TimeFrameUnit.Minute
)

symbol = 'TSLA'

data_request = StockBarsRequest(
    symbol_or_symbols=symbol, 
    start = datetime.now() - timedelta(days = 1), 
    end = datetime.now(), 
    timeframe = timeframe 
)

historical_data = historical_data_client.get_stock_bars(request_params=data_request)

In [7]:
historical_data_df = historical_data.df.droplevel(level= 0)

In [8]:
ta.rsi(close = historical_data_df['close']).dropna()

timestamp
2022-12-19 19:22:00+00:00    45.265138
2022-12-19 19:24:00+00:00    45.882611
2022-12-19 19:26:00+00:00    48.378199
2022-12-19 19:28:00+00:00    34.082191
2022-12-19 19:30:00+00:00    42.050763
                               ...    
2022-12-20 18:44:00+00:00    37.672388
2022-12-20 18:46:00+00:00    36.834041
2022-12-20 18:48:00+00:00    37.305056
2022-12-20 18:50:00+00:00    40.571064
2022-12-20 18:52:00+00:00    37.916067
Name: RSI_14, Length: 457, dtype: float64

In [9]:
calendar_request = GetCalendarRequest(
    start = datetime.now() - timedelta(days = 10), 
    end = datetime.now()
)

days = trading_client.get_calendar(calendar_request)

In [10]:
trading_day = days[0]